In [ ]:
#PLEASE RUN THIS CELL 
import requests
from IPython.core.display import HTML


In [ ]:
# Import necessary libraries

import gym
import time
import random
import numpy as np
from gym import Env
import tensorflow as tf
import matplotlib.image as img
import matplotlib.pyplot as plt
from collections import namedtuple
from gym.spaces import Discrete, Box 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Input, Flatten, MaxPooling2D, Add


%matplotlib inline

### INSTRUCTIONS


- This homework is a jupyter notebook. Download and work on it on your local machine.

- This homework should be submitted in pairs.

- Ensure you and your partner together have submitted the homework only once. Multiple submissions of the same work will be penalised and will cost you 2 points.

- Please restart the kernel and run the entire notebook again before you submit.

- Running cells out of order is a common pitfall in Jupyter Notebooks. To make sure your code works restart the kernel and run the whole notebook again before you submit. 

- To submit the homework, either one of you upload the working notebook on edStem and click the submit button on the bottom right corner.

- Submit the homework well before the given deadline. Submissions after the deadline will not be graded.

- We have tried to include all the libraries you may need to do the assignment in the imports statement at the top of this notebook. We strongly suggest that you use those and not others as we may not be familiar with them.

- Comment your code well. This would help the graders in case there is any issue with the notebook while running. It is important to remember that the graders will not troubleshoot your code. 

- Please use .head() when viewing data. Do not submit a notebook that is **excessively long**. 

- In questions that require code to answer, such as "calculate the $R^2$", do not just output the value from a cell. Write a `print()` function that includes a reference to the calculated value, **not hardcoded**. For example: 
```
print(f'The R^2 is {R:.4f}')
```
- Your plots should include clear labels for the $x$ and $y$ axes as well as a descriptive title ("MSE plot" is not a descriptive title; "95 % confidence interval of coefficients of polynomial degree 5" is).

- **Ensure you make appropriate plots for all the questions it is applicable to, regardless of it being explicitly asked for.**

<hr style="height:2pt">

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
## **PART 1 [35 points]: Custom Environment**
<br />    

In the first part of the homework, you are expected to define a custom environment. Based on the definition, you will implement policy iteration to compare different policies.
    
    
</div>
    

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

## **PART 1: Questions**
<br />

**ENVIRONMENT DESCRIPTION:**

We have a hallway consisting of 5 blocks (states 0-4). There are two actions, which deterministically move the agent to the left or the right. More explicitly: Performing action “left” in state 0 keeps you in state 0, moves you from state 1 to state 0, from state 2 to state 1, state 3 to state 2, and state 4 to state 3. Performing action “right” in state 4 keeps you in state 4, moves you from state 3 to state 4, from state 2 to state 3, from state 1 to state 2, and from state 0 to state 1. The agent receives a reward of -1.0, if any action causes the agent to move to or remain at state 0, state 1, state 2, or state 3. The agent receives a reward of +10.0 if it goes to state 4. If the agent remains in state 4 for 3 consequtive steps, end the episode. Let the discount factor, γ = 0.75.

Set the default initial state of the environment to be 0.
    
<img src="image/grid.png" alt="Model Input" style="width:500px">
<br /><br />

### **1.1 [20 points] ENVIRONMENT DEFINITION**
<br />

**1.1.1** - Based on the environment definition, implement the custom environment with appropriate states, rewards and state transitions. Name this environment as `HallwayEnvironment`.
<br /><br />

**1.1.2** - The probability of moving in the direction as specified by the selected action is given 0.75 and the probability of moving in the other direction is 0.25. Define a probability transition matrix that gives the probability of going to one state from the given state. 
<br /><br />

**1.1.3** - Define a reward matrix that gives the reward for being in a particular state. This is -1 for all states except the last one.
<br /><br />

### **1.2 [15 points] POLICY ITERATION**
<br />

**1.2.1**  Policy Evaluation: Initialize a policy “left” for every state. Implement policy evaluation as described in the lecture. That is, for each possible starting state, what is the expected sum of future rewards for this policy i.e. the value function? 
<br /><br />

**1.2.2**  Policy Evaluation: Initialize a policy “random” for every state. Implement policy evaluation as described in the lecture. That is, for each possible starting state, what is the expected sum of future rewards for this policy i.e. the value function? 
<br /><br />

**1.2.3** Provide a comparision of the 2 policies computing their value functions.
<br /><br />
    
**1.2.4** Policy Iteration: Implement policy iteration. Report the sequence of policies you find starting with both the policies defined above in every state. Do they converge to the same policy? 
<br /><br />

</div>

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
## **PART 1: Solutions**
<br />

**ENVIRONMENT DESCRIPTION:**

We have a hallway consisting of 5 blocks (states 0-4). There are two actions, which deterministically move the agent to the left or the right. More explicitly: Performing action “left” in state 0 keeps you in state 0, moves you from state 1 to state 0, from state 2 to state 1, state 3 to state 2, and state 4 to state 3. Performing action “right” in state 4 keeps you in state 4, moves you from state 3 to state 4, from state 2 to state 3, from state 1 to state 2, and from state 0 to state 1. The agent receives a reward of -1.0 if it starts any iteration in state 0, state 1, state 2, or state 3. The agent receives a reward of +10.0 if it starts in state 4. If the agent remains in state 4 for 3 consequtive steps, end the episode.  Let the discount factor γ = 0.75.

Set the default initial state of the environment to be 0. However, this can change. The agent can choose to start in a different state.

<img src="image/grid.png" alt="Model Input" style="width: 500px">
<br /><br />


### **1.1 [20 points] ENVIRONMENT DEFINITION**
<br />

**1.1.1** - Based on the environment definition implement the custom environment with appropriate states, rewards and state transitions. Name this environment as `HallwayEnvironment`.
<br />
    
</div>

In [ ]:
class HallwayEnvironment(Env):
  
  # The init function of the environment class
  def __init__(self):

    # The possible observation space is discrete 5 values
    self.observation_space = Discrete(5)

    # Possible actions are left and right
    # Action 0 - LEFT
    # Action 1 - RIGHT
    self.action_space = Discrete(2)
    
    # The initial state of the agent in the environment
    self.state = 0
    
    # Initial reward is 0
    self.reward = 0

    # Check if episode ended
    self.done = 0
    
  def step(self, action):

    # Set the action of the environment from the method parameter
    self.action = action

    # If action is left
    if self.action==0:
      if self.state==0:
        next_state = self.state
      elif self.state>0 and self.state<=4:
        next_state = self.state - 1
    
    # If action is right
    if self.action==1:
      if self.state==4:
        next_state = self.state
      elif self.state<4 and self.state>=0:
        next_state = self.state + 1

    if self.state==4:
      self.reward+=10
      self.done+=1
    else:
      self.reward-=1
      self.done=0

    if self.done==3:
      done = True
    else:
      done = False

    self.state = next_state

    return self.state, self.reward, done



<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
**1.1.2** - The probability of moving in the direction as specified by the selected action is given 0.75 and the probability of moving in the other direction is 0.25. Define a probability transition matrix that gives the probability of going to one state from the given state. 
<br />

</div>

In [ ]:
# Get the environment
env = HallwayEnvironment()

# Get the number of actions
n_actions = env.action_space.n

# Get the number of states
n_states = env.observation_space.n

# The discount rate
gamma = 0.75

In [ ]:
# Function to get the next state given the current state and action
def get_state(state, action):

  # If action is left
  if action==0:
    if state==0:
      next_state = state
    elif state>0 and state<=4:
      next_state = state - 1
  
  # If action is right
  if action==1:
    if state==4:
      next_state = state
    elif state<4 and state>=0:
      next_state = state + 1  

  return next_state

In [ ]:
# The action taken, the start state, the resultant state 
transition_probability = np.zeros(shape=(n_actions,n_states,n_states))

# Loop over the number of actions
for i in range(n_actions):

  # Loop over the number of states
  for j in range(n_states):
    action = i
    state = get_state(j, i)
    transition_probability[action,j,state] = 0.75

    # To set the probability in case of the opposite action
    # The next state changes and hence we call the function again
    if i==0:
      action = 1
      transition_probability[action,j,state] = 0.25
    else:
      action = 0
      transition_probability[action,j,state] = 0.25

print("STATE TRANSITION PROBABILITIES\n\n", transition_probability)

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**1.1.3** - Define a reward matrix that gives the reward for being in a particular state. This is -1 for all states except the last one.
<br />

</div>

In [ ]:
reward_matrix = transition_probability

reward_matrix = np.where(reward_matrix!=0, -1, reward_matrix)

for i in range(2):
  reward_matrix[i][3:5][0][4]=10
  reward_matrix[i][3:5][1][4]=10

reward_matrix

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
### **1.2 [15 points] POLICY ITERATION**
<br />

**1.2.1**  Policy Evaluation: Initialize a policy “left” for every state. Implement policy evaluation as described in the lecture. Get for each possible starting state, what is the expected sum of future rewards for this policy? 
<br />

</div>

In [ ]:
# Initializing a left policy.
left_policy = np.zeros(shape=(n_states, n_actions), dtype=int)
left_policy[:,0] = 1
left_policy


In [ ]:
# Function to compute the value function
# This function takes as input the policy and discount factor

def policy_evaluation(policy, gamma):
    
    # Initialize the value table with zeros
    # The size of the value table is the equal to the number of states
    value_table = np.zeros(n_states)
    
    # Loop for a few iterations
    for l in range(75):
        
        # Save the value table to the updated_value_table
        updated_value_table = value_table

        # For each state in the environment, 
        # select the action according to the policy and compute the value table
        for state in range(n_states):

            # Get the action from the policy based on the state
            action = np.argmax(policy[state])
            
            # For the selected action, compute the value function
            value_table[state] = np.sum([transition_probability[action][state][i]* ( reward_matrix[action][state][i] + gamma * updated_value_table[get_state(state, action)]) for i in range(len(transition_probability[action][state]))])

    # Return the value table
    return value_table


In [ ]:
value_function = policy_evaluation(left_policy, gamma)
print("Value Function of Left Policy\n\n", value_function)


<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**1.2.2**  Policy Evaluation: Initialize a policy “random” for every state. Implement policy evaluation as described in the lecture. That is, for each possible starting state, what is the expected sum of future rewards for this policy i.e. the value function? 

<br /><br />

    
</div>

In [ ]:
# Initializing a random policy.
random_policy = np.zeros(shape=(n_states, n_actions), dtype=int)
for i in range(len(random_policy)):
    random_policy[i,0] = np.random.choice([0,1])
    random_policy[i,1] = not random_policy[i,0]

random_policy

In [ ]:
value_function = policy_evaluation(random_policy, gamma)
print("Value Function of Random Policy\n\n", value_function)


<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
**1.2.3** Policy Iteration: Implement policy iteration. Report the sequence of policies you find starting with both the policies defined above in every state. Do they converge to the same policy? 
<br />

</div>

In [ ]:
# Function to extract the optimal policy based on the optimal value function
# The function takes as input the value table and the discount factor

def policy_improvement(value_table, policy, gamma = 1.0):
 
    # Initialize the policy with zeros
    # The size of the policy is equivalent to the number of states
    policy = policy 
    
    # Loop for each state 
    for state in range(n_states):
        
        # Initialize the Q table for a state
        # with zeroes and the number of possible actions
        Q_table = np.zeros(n_actions)
        
        # Loop for each action
        for action in range(n_actions):
            
            # For the given state and action, loop over the next states
            for i in range(len(transition_probability[action][state])): 
                
                # Get the transition probability, next state, reward from the the environment
                next_state = get_state(state, action)
                trans_prob = transition_probability[action][state][i]
                reward = reward_matrix[action][state][i]
                
                # Update the q-table the particular action based on the equation in the instructions
                Q_table[action] += (trans_prob * (reward + gamma * value_table[next_state]))
        
        # Select the action which has maximum Q value as an optimal action of the state
        policy[state][np.argmax(Q_table)] = 1
        policy[state][np.argmax(Q_table) ^ 1] = 0
    
    # Return the policy
    return policy


In [ ]:
# Function to perform policy iteration
# This function takes the environment and discount factor

def policy_iteration(policy ,gamma = 1.0):
    
    # Initialize policy with zeros for the dimension of the number of state
    old_policy = policy
    
    # Specify the number of iterations
    no_of_iterations = 5
    
    # Loop over the number of iterations
    for i in range(no_of_iterations):
        
        # Compute the value function calling the policy_evaluation 
        # by passing the policy and discount factor
        new_value_function = policy_evaluation(old_policy, gamma)
        
        # Extract the new policy by calling the policy_improvement function with the 
        # new value function and the discount factor
        new_policy = policy_improvement(new_value_function, old_policy, gamma)
   
        # Check whether we have reached convergence i.e whether we found the optimal
        # policy by comparing old_policy and new policy.
        # If the policies are the same, break the loop, else update the old policy
        old_policy = new_policy
        
    return new_policy



In [ ]:
print(policy_iteration(left_policy, gamma))


In [ ]:
print(policy_iteration(random_policy, gamma))

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
## **PART 2 [65 points]: Deep Q Learning**
<br />    

In this part of the homework, you will work with the Breakout-v0 environment from OpenAI Gym. You will learn to play the game using a modified version of Deep Q Learning. 
    
    
</div>

In [ ]:
# Helper code to get the ROMS file to work with Atari Environments directly on colab
!gdown --id 1HhnGSXKgivZN4wGzrZCnjKAUAOJ18QuA
!unzip Roms.zip
!unzip -qq Roms/ROMS.zip
!unzip -qq Roms/HC\ ROMS.zip


In [ ]:
!python -m atari_py.import_roms ROMS


<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

## **PART 2: Questions**
<br />

### **2.1 [10 points] PRE-PROCESSING**
<br />

Load the `Breakout-v0` environment from OpenAI Gym. Take a look at the possible actions and the state of the environment. What does each action integer represent? Describe the state of the environment. What does it represent?

<br />

### **2.2 [55 points] DQN Network**
<br />


**2.2.1** Define a replay memory to store the experiences which we will use to train the model later on.
<br /><br />

**2.2.2** Define a policy to take an action in each step of the environment.
<br /><br />

**2.2.3** Implement a DQN network which takes as input the state of the environment and returns the Q-value associated with each action you can take in that state. The Q-value is a sum of the value function and an advantage function. 

The architecture of the network is given in the image below.
<img src="image/hw.png" alt="Model Input" style="width:700px">
    
<br /><br />

**2.2.4** Train the model with a policy and target network as discussed in the lecture.
<br /><br />

**2.2.5** Test the network by allowing your agent in the environment for 2 episodes and displaying the reward at the end of each episode.
<br />
</div>

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

## **PART 2: Solutions**
<br />

### **2.1 [10 points] PRE-PROCESSING**
<br />

Load the `Breakout-v0` environment from OpenAI Gym. Take a look at the possible actions and the state of the environment. What does each action integer represent?

<br />

</div>

In [ ]:
# Initialize the gym environment
env = gym.make('Breakout-v0')

# Get information about the action and state space
n_actions = env.action_space.n
n_states = env.observation_space.shape

print("The state space is:", n_states)
print("The number of actions is:", n_actions)


In [ ]:
n_states = (210,160,1)


In [ ]:
# Meaning of each action in the environment
env.env.get_action_meanings()

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

### **2.2 [55 points] DQN Network**
<br />


**2.2.1** Define a replay memory to store the experiences which we will use to train the model later on.


</div>

In [ ]:
# Creating a namedtuple class Experience that will hold the state, action, next state and reward
Experience = namedtuple('Experience', ('state', 'action', 'next_state', 'reward'))


In [ ]:
# Class ReplayMemory to add experience to our class instance and
# sample memory 
class ReplayMemory():

  # __init__ function to initialize the memory capacity
  def __init__ (self, capacity):
    self.memory_capacity = capacity
    self.memory = []
    self.push_count = 0

  # Method to add memory instance
  def push(self, experience):
    if len(self.memory) < self.memory_capacity:
      self.memory.append(experience)
    else:
      self.memory[self.push_count % self.capacity] = experience
    self.push_count+=1

  # Method to sample from memory
  def sample(self, batch_size):
    return random.sample(self.memory, batch_size)



<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.2.2** Define a policy to take an action in each step of the environment.
<br />

</div>

In [ ]:
# Define a function epsilon greedy that takes as parameter, the neural network,
# state and episode number
def epsilon_greedy(policy_network, state, episode):

  # Compute a random threshold value
  threshold = np.random.uniform(0,1)

  # Compute the epsilon value (exploration rate) based on the equation in the isntructions
  epsilon = eps_min + (eps_max-eps_min) * np.exp(-eps_decay_rate*episode)

  # Check if the threshold value is lower than the updated epsilon
  if threshold < epsilon:

      # Take a random action
      action = env.action_space.sample()

  # Else take the best action for that state by selecting the one that gives the
  # highest q-value when given as input to the neural network
  else:
      action = np.argmax(state)

  # Return the action
  return action


<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.2.3** Implement a DQN network which takes as input the state of the environment and returns the Q-value associated with each action you can take in that state. The Q-value is a sum of the value function and an advantage function. 
    
The network architecture is given in the following figure.
<img src="image/hw.png" alt="Model Input" style="width:700px">


<br/>

</div>

In [ ]:
# Function to return the convolutional neural network for DQN
def get_network(name):

    input_shape = (n_states)

    # Define size of the pooling operation
    pool_size=(2,2)

    inp = Input(shape=n_states, name="Input")
    l1 = Conv2D(256, kernel_size=(3,3), activation='relu', name="Conv1")(inp)
    l2 = Conv2D(128, kernel_size=(3,3), activation='relu', name="Conv2")(l1)
    m1 = MaxPooling2D(pool_size=pool_size, name="Pool1")(l2)
    l3 = Conv2D(64, kernel_size=(3,3), activation='relu', name="Conv3")(m1)
    m2 = MaxPooling2D(pool_size=pool_size, name="Pool2")(l3)
    l4 = Conv2D(32, kernel_size=(3,3), activation='relu', name="Conv4")(m2)
    m2 = MaxPooling2D(pool_size=pool_size, name="Pool3")(l4)
    flat = Flatten()(m2)

    # Code for value function computation
    d1 = Dense(256, activation='relu', name="Dense1")(flat)
    d2 = Dense(1, activation='linear', name="Dense2")(d1)

    # Code for advantage computation
    d3 = Dense(256, activation='relu', name="Dense3")(flat)
    d4 = Dense(n_actions, activation='linear', name="Dense4")(d3)

    y = Add(name="Output")([d2, d4])

    model = Model(inputs=inp, outputs=y, name=name)
    return model


<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.2.4** Train the model with a policy and target network as discussed in the lecture.
<br />

</div>

In [ ]:
# Initialize the policy and target networks
policy_network = get_network("Policy")

target_network = get_network("Target")

target_network.set_weights(policy_network.get_weights())


In [ ]:
target_network.summary()

In [ ]:
# Set the batch size
batch_size = 256

# Set the discount rate
gamma = 0.99

# Set the minimum discount rate
eps_min = 0.01

# Set the maximum discount rate
eps_max = 1

# Set the epsilon decay rate
eps_decay_rate = 0.001

# Set the number of updates to the policy network after
# which we update the target network
target_update = 100

# Set capacity of replay memory
memory_size = 210*160*500

# Set learning rate of policy network
lr = 0.001

# Set the number of episodes
num_episodes = 5

# Warm-up steps - the number of experiences to store before
# model training begins
# It has to be atleast one more than the batch size
warm_up_steps = batch_size*2

# Set the number of steps in each episode
steps_per_episode = 100

# Variable to store the total number of steps in all episodes
global_step_count = 0

In [ ]:
# Create an instance of the ReplayMemory class by specifying the 
# memory capacity
memory = ReplayMemory(memory_size)


In [ ]:
# Define the optimizer to train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Define the loss function of the model
loss_fn = tf.keras.losses.MeanSquaredError()



In [ ]:
# Loop over all the episodes
for episode in range(1,num_episodes+1):

  # Print the spideo number
  print("Episode:", episode)

  # Get the initial state of the environment  
  state = env.reset()
  state = state.mean(axis=2)

  # Initialize a counter to update the target network
  update_count = 0

  # Loop over the maximium number of steps in the environment
  for step in range(steps_per_episode):

    # Increment the total step count variable
    global_step_count+=1

    # Get the action based on the epsilon greedy policy
    action = epsilon_greedy(policy_network, state, episode)

    # Get the next state, reward, whether or not the episode is done 
    # by calling the step method with the action
    if action not in range(4):
      action = np.random.choice(range(4))
    
    next_state, reward, done, info = env.step(action)
    next_state = next_state.mean(axis=2)

    # Push this experience to the replay memory by calling the push method 
    # of the ReplayMemory class with the appropriate parameters
    memory.push(Experience(state, action, next_state, reward))

    # Update the current state of the environment
    state = next_state

    # Check if the total number of steps is greater than the warm-up steps to begin training
    if global_step_count > warm_up_steps:

      # Increment the counter to update the target network
      update_count+=1

      # Sample experiences from the memory
      experiences = np.asarray(memory.sample(batch_size))

      # Split the experience into a list of states, actions, next_states and rewards
      states, actions, next_states, rewards = experiences[:,0].tolist(), experiences[:,1].tolist(), experiences[:,2].tolist(), experiences[:,3].tolist()

      # Compute the target q-values from the target network by passing the next states 
      next_q_values = [tf.cast(tf.math.reduce_max(tf.cast(tf.math.reduce_max(target_network(np.array(next_states[i]).reshape((-1,)+n_states))), dtype=tf.float32)), dtype=tf.float32) for i in range(batch_size)]

      target_q_values = [(next_q_values[i]*gamma)+ rewards[i] for i in range(batch_size)] 

      # Initialize a gradient tape
      with tf.GradientTape() as tape:

        # Get the predicted q-values from the policy network by passing the current states and the specific action from memory
        predicted_q_values = [tf.cast(tf.math.reduce_max(policy_network(np.array(states).reshape((-1,)+n_states))), dtype=tf.float32)[int(actions[i])] for i in range(batch_size)]
        
        # Compute the loss between the target q-values and the predicted q-values
        loss = loss_fn(target_q_values, predicted_q_values)

      # Get the gradients wrt the policy network
      gradients = tape.gradient(loss, policy_network.trainable_weights)

      # Update the weights of the policy network
      optimizer.apply_gradients(zip(gradients, policy_network.trainable_weights))
    
    # If the update counter has reached the update threshold defined as target_update above
    if update_count%target_update==0:


      # Update the weights of the target network with that of the policy network
      target_network.set_weights(policy_network.get_weights())

    # If done is True, then end the episode
    if done==True:
      break



<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.2.5** Test the network by allowing your agent in the environment for 2 episodes and displaying the reward at the end of each episode.

</div>

In [ ]:
# Get the initial state of the environemnt
state = env.reset()

# Set done as False
done = False

reward = 0

# Initialize a counter variable to keep track of the number of steps
# before the agent reached the goal or falls into a hole
count = 0

# Loop over the episode
while done!=True:
    state = state.sum(axis=2)
    # Get the action for the given state from the policy network
    action = np.argmax(policy_network(np.array(state).reshape((-1,)+n_states)))
    print(action)

    # Get the next state and reward from the environment for the particular action taken
    next_state, r, done, info = env.step(action) 

    # Print the state and action taken
    print("State:", state, "\tAction:", action)

    # Update the current state
    state = next_state

    # Increment the step counter
    count+=1

    reward+=r

print("Game ended after:", count," steps with a reward of", reward)